# Session 3: Beyond Variational Quantum Optimization

In the last session, we saw how to use variational algorithms such as VQE or QAOA to solve optimization problems using quantum computers.

But VQE and QAOA require variational loops, and it's not exactly clear how ansatz selection, parameter optimization, etc. impact solution accuracy. Further, whether these algorithms will scale up is non-obvious.

What are we to do?

Thankfully, there are algorithms for quantum optimization which do not rely on variational methods. We will explore one, Grover adaptive search, [here](https://arxiv.org/abs/1912.04088).

## From last time: MaxCut

In [ ]:
# Some standard code imports
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np

# For drawing graphs
import networkx as nx

# Qiskit imports
from qiskit import Aer, execute, QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.optimization.converters import LinearEqualityToPenalty
from qiskit.optimization import QuadraticProgram

# auxilliary function to plot graphs
def plot_result(G, x):
    colors = ['r' if x[i] == 0 else 'b' for i in range(n)]
    pos, default_axes = nx.spring_layout(G), plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, pos=pos)

In [ ]:
#-------------------------MAKING THE GRAPH---------------------------#
# Create graph
G = nx.Graph()

# Add 5 nodes
n = 5
G.add_nodes_from(range(n))

# Add edges: tuple is (i,j,weight) where (i,j) is the edge
edges = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0), (2, 4, 1.0), (3, 4, 1.0)]
G.add_weighted_edges_from(edges)

#-----------------------WRITING A DOCPLEX MODEL--------------------#
# Import a model from DOcplex
from docplex.mp.model import Model

# Name the model
mdl = Model('MaxCut')

# Add a binary variable to the model for each node in the graph
x = mdl.binary_var_list('x{}'.format(i) for i in range(n))

# Define the objective function
objective = mdl.sum([ w * (x[i] + x[j] - 2*x[i]*x[j]) for (i, j, w) in edges])

# And let's maximize it!
mdl.maximize(objective)

# Add an equality constraint
b = 2
mdl.add_constraint(mdl.sum(x) == b)

#--------------------CONVERSION TO ISING, VIA QUADRATICPROGRAM-----#
# Set up the quadratic program
qp = QuadraticProgram()

# Put the model inside it
qp.from_docplex(mdl)

# Convert the program to a QUBO
qp_eq = LinearEqualityToPenalty(penalty=1).convert(qp)

## Grover Search

Consider a boolean function $f: \{0, \ldots, 2^n-1\} \rightarrow \{0, 1\}$ and an oracle $U_f$ such that

$$
U_f: |x\rangle_n \rightarrow (-1)^{f(x)} |x\rangle_n
$$

Grover Search allows to find an $x \in \{0, \ldots 2^n-1\}$ with $f(x)=1$ using only $\mathcal{O}(\sqrt{2^n})$ queries to the oracle, while the best classical approach requires $\mathcal{O}(2^n)$ queries to $f$.

<div align="center">
<img src="https://qiskit.org/textbook/ch-algorithms/images/grover_circuit_high_level.png" width="800"/>
Source: https://qiskit.org/textbook/ch-algorithms/grover.html
</div>

## Grover Adaptive Search

Grover Adaptive Search (GAS) has been explored as a possible solution for combinatorial optimization problems, alongside variational algorithms such as Variational Quantum Eigensolver (VQE) and Quantum Approximate Optimization Algorithm (QAOA). The algorithm iteratively applies Grover Search to find the optimum value of an objective function, by using the best-known value from the previous run as a threshold. The adaptive oracle used in GAS recognizes all values above or below the current threshold (for max and min respectively), decreasing the size of the search space every iteration the threshold is updated, until an optimum is found.

Consider a function $f: \{0, \ldots, 2^n-1\} \rightarrow \mathbb{Z}$, an input $x' \in \{0, \ldots, 2^n-1\}$, and an oracle $U_{f}(x')$ such that

$$
U_f(x'): |x\rangle_n \rightarrow (-1)^{f(x) < f(x')} |x\rangle_n
$$

Grover Adaptive Search works as follows:
1. Run Grover Search to find an $x$ with $f(x) < f(x')$.
2. Set $x' = x$ and repeat until no further improvement is made.

This algorithm needs three ingredients:

1. A state preparation operator $A$ to construct a superposition of all states in the search space.

2. An oracle operator $O$, that recognizes the states of interest and multiplies their amplitudes by -1.

3. The Grover diffusion operator $D$, that multiplies the amplitude of the $|0\rangle_n$ state by -1.

## Building block 1 : Computing the [two's complement](https://en.wikipedia.org/wiki/Two%27s_complement)

In [ ]:
# Converts two's complement bit string to corresponding integer
def twos_complement(val, num_bits):
    val = int(val, 2)
    if (val & (1 << (num_bits - 1))) != 0:
        val = val - (1 << num_bits)     
    return val   

In [ ]:
print(twos_complement('0000', 4))
print(twos_complement('0101', 4))
print(twos_complement('1010', 4))
print(twos_complement('1111', 4))

## Building block 2: Phase encoding and the Quantum Fourier Transform (QFT)
For fixed $k \in \{0, \ldots, 2^n-1\}$, and starting in $|+\rangle_n$, we can construct

$$
\frac{1}{\sqrt{2^n}}\sum_{x=0}^{2^n-1} e^{2\pi i \frac{xk}{2^n}} |x\rangle_n = 
\bigotimes_{i=0}^{n-1} R_Z\left(2\pi \frac{2^i}{2^n} k\right) |+\rangle_n =: U_k |+\rangle_n.
$$

This implies

$$
QFT^{-1} U_k |+\rangle_n = |k\rangle_n.
$$

Furthermore, for fixed $k_1, k_2$ we get 

$$
QFT^{-1} U_{k_2} U_{k_1} |+\rangle_n = |k_1 + k_2\rangle_n.
$$

That is, by using the phase encoding and the inverse QFT, we can add integers.

In [ ]:
# Import the QFT from Qiskit's circuit library
from qiskit.circuit.library import QFT

# Other Qiskit imports
from qiskit import QuantumCircuit, execute, Aer

import numpy as np

In [ ]:
# A function to encode an integer into a number
# of qubits using the phase encoding.
def encode(num_qubits, k):
    qc = QuantumCircuit(num_qubits, name='enc({})'.format(k))
    for j in range(num_qubits):
        # Angle of rotation
        theta = 2*np.pi * 2**j / 2**num_qubits * k
        qc.rz(theta, j)
    return qc

In [ ]:
# Encode the number 2 in 4 qubits
encode(4, 2).draw()

In [ ]:
num_value_qubits = 4
number_to_encode = 10

qc = QuantumCircuit(num_value_qubits, num_value_qubits)
qc.h(range(num_value_qubits))
qc.barrier()
qc.append(encode(num_value_qubits, number_to_encode), range(num_value_qubits))
qc.barrier()
qc.append(QFT(num_value_qubits, do_swaps=False).inverse(), qc.qubits)

# Note: we have to mess around with the bitstring ordering here
# in order for the two's compliment math to work out.
qc.measure(qc.qregs[0], qc.cregs[0][::-1])

qc.draw(fold=120)

Let's simulate this circuit. Using the two's compliment representation of the bitstring, we can check whether the number is correct.

In [ ]:
counts = execute(qc, Aer.get_backend('qasm_simulator')).result().get_counts()
for key in counts:
    print(key, ' -->', twos_complement(key, num_value_qubits))

Indeed, from the graphic in the talk, we see that the two's compliment representation of 10 is -6.

We can also show that this circuit correctly enables us to do addition.

In [ ]:
num_value_qubits = 4
qc = QuantumCircuit(num_value_qubits, num_value_qubits)
qc.h(range(num_value_qubits))
qc.barrier()
# Encode the number 2
qc.append(encode(num_value_qubits, 2), range(num_value_qubits))
# Encode the number 10
qc.append(encode(num_value_qubits, 10), range(num_value_qubits))
qc.barrier()
qc.append(QFT(num_value_qubits, do_swaps=False).inverse(), qc.qubits)
qc.measure(qc.qregs[0], qc.cregs[0][::-1])
qc.draw(fold=120)

In [ ]:
counts = execute(qc, Aer.get_backend('qasm_simulator')).result().get_counts()
for key in counts:
    print(key, ' -->', twos_complement(key, num_value_qubits))

The two's compliment representation of 2 + 10 = 12 is indeed -4, so the circuit is working.

## Building block 3: circuits to evaluate quadratic forms

Qiskit's circuit library comes with a [circuit that evaluates quadratic forms](https://qiskit.org/documentation/stubs/qiskit.circuit.library.QuadraticForm.html) (using the two's compliment representation). We can use this object to directly encode the quadratic form we're trying to optimize.

In [ ]:
from qiskit.circuit.library import QuadraticForm

In [ ]:
# get quadratic / linear / constant part of quadratic program
A = qp_eq.objective.quadratic.to_array()
b = qp_eq.objective.linear.to_array()
c = qp_eq.objective.constant

# set number of results qubits
num_value_qubits = 5

# construct circuit to evaluate quadratic form
qf = QuadraticForm(num_value_qubits, A, b, c)
qf.draw(fold=120)

So now we have a circuit that can evaluate the objective function. Notice that while the our input problem requires 5 qubits (the graph has 5 nodes), the circuit itself has 10 qubits, since we're using 5 qubits to represent the values.

Let's use this circuit to evaluate the objecive function on the superposition of $2^5$ bitstrings.

In [ ]:
# Make a big circuit on 10 qubits
qc = QuantumCircuit(n + num_value_qubits)

# Initialize the data qubits to be a superposition state
qc.h(range(n))

# Add the circuit which evaluates the objective function
qc.append(qf, range(n + num_value_qubits))
qc.measure_all()
qc.draw()

Let's simulate the circuit. The code block below grabs the first 5 bits as the node assignment.

In [ ]:
counts = execute(qc, backend=Aer.get_backend('qasm_simulator')).result().get_counts()
for key, value in counts.items():
    x_ = key[num_value_qubits:]
    x = [0 if x__ == '0' else 1 for x__ in x_][::-1]
    y_bin = key[:num_value_qubits]
    y_int = twos_complement(y_bin, num_value_qubits)
    qx = qp_eq.objective.evaluate(x)    
    print('x =', x_, '\ty_bin =', y_bin, '\ty_int =', y_int, '\tQ(x) =', qx, '\t(counts: {})'.format(value))

## Creating the Grover oracle

Now that we have a circuit which can compute the objective function, we can create the Grover oracle, which needs to mark states where the objective function is less than 0.

In [ ]:
qc = QuantumCircuit(n + num_value_qubits, name='U_f')
qc.append(qf, range(n + num_value_qubits))            # 1. compute Q(x)
qc.z(qc.qubits[-1])                                   # 2. multiply all |x>|Q(x)> by -1 where Q(x) < 0.
qc.append(qf.inverse(), range(n + num_value_qubits))  # 3. uncompute Q(x).
qc.draw()

In [ ]:
qc_grover = QuantumCircuit(n + num_value_qubits)
qc_grover.h(range(n))
qc_grover.append(qc, range(qc_grover.num_qubits))
qc_grover.draw()

If we run this circuit, we'll find that the amplitudes associated with bitstrings $x$ where $Q(x) < 0$ are negative.

In [ ]:
data = Statevector.from_instruction(qc_grover).data
x = ['{0:05b}'.format(i) for i in range(2**n)]
y = [qp_eq.objective.evaluate([0 if x__ == '0' else 1 for x__ in reversed(x_)]) for x_ in x]
plt.figure(figsize=(12, 5))
plt.bar(range(2**n), np.real(data[:2**n]))
plt.xticks(range(2**n), ['{} $\leftarrow$ '.format(y[i]) + '{0:05b}'.format(i) for i in range(2**n)], rotation=90, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

Next, we need a reflection operator which reflects the state about $|00000>$.

In [ ]:
reflection = QuantumCircuit(n, name='reflection')
reflection.h(range(reflection.num_qubits))
reflection.barrier()
reflection.x(range(reflection.num_qubits))
reflection.barrier()
reflection.h(-1)
reflection.mct(list(range(reflection.num_qubits - 1)), -1)
reflection.h(-1)
reflection.barrier()
reflection.x(range(reflection.num_qubits))
reflection.barrier()
reflection.h(range(reflection.num_qubits))
reflection.draw()

Let's tack this reflection on to the Grover circuit.

In [ ]:
qc_grover.append(reflection, range(n))
qc_grover.draw()

In [ ]:
data = Statevector.from_instruction(qc_grover).data
x = ['{0:05b}'.format(i) for i in range(2**n)]
y = [qp_eq.objective.evaluate([0 if x__ == '0' else 1 for x__ in reversed(x_)]) for x_ in x]
plt.figure(figsize=(12, 5))
plt.bar(range(2**n), -np.real(data[:2**n]))  # multiply by -1, since reflection is implemented up to global phase -1
plt.xticks(range(2**n), ['{} $\leftarrow$ '.format(y[i]) + '{0:05b}'.format(i) for i in range(2**n)], rotation=90, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

The reflection has boosted the amplitude of some states, and decreased it for others.

Instead of having to construct all these circuits yourself, Qiskit provides a helper function for Grover adaptive search.

In [ ]:
from qiskit.optimization.algorithms import GroverOptimizer

In [ ]:
# set up Grover Optimizer
grover = GroverOptimizer(num_value_qubits=5, quantum_instance=Aer.get_backend('statevector_simulator'))
grover._qubo_converter.penalty = 1  # set to small value to reduce required number of value qubits

# solver problem
result = grover.solve(qp)

In [ ]:
print(result)
plot_result(G, result.x)